# `DataFrames`, `QTables`, and units

In [1]:
import numpy as np
import pandas as pd

from astropy import units as u
from astropy import constants as const

In [2]:
import astropy

astropy.__version__

'4.0'

In [3]:
comet_table = pd.read_csv('./Data/Comets.csv')

In [4]:
comet_table

,Name,Semi_Major_AU,Eccentricity
0,1P/Halley,17.834144,0.967143
1,2P/Encke,2.215141,0.848336
2,3D/Biela,3.534658,0.751299
3,4P/Faye,3.838159,0.569618


### `DataFrames` and units - An Example

* Use `.values` to pull data out of `DataFrame`
* Then add the units

In [5]:
semi_major = comet_table['Semi_Major_AU'].values * u.AU

In [6]:
semi_major

<Quantity [17.83414429,  2.21514129,  3.53465808,  3.83815916] AU>

In [7]:
semi_major.to(u.km)

<Quantity [2.66795001e+09, 3.31380421e+08, 5.28777323e+08, 5.74180437e+08] km>

In [8]:
comet_table['Semi_Major_km'] = semi_major.to(u.km)

In [9]:
comet_table

,Name,Semi_Major_AU,Eccentricity,Semi_Major_km
0,1P/Halley,17.834144,0.967143,2.667950e+09
1,2P/Encke,2.215141,0.848336,3.313804e+08
2,3D/Biela,3.534658,0.751299,5.287773e+08
3,4P/Faye,3.838159,0.569618,5.741804e+08


#### Pull out `.values` and assign units everytime you want to use them.

* Even with dimensionless units (like `Eccentricity`).

In [10]:
def find_perihelion(semi_major, eccentricity):
    result = semi_major * (1.0 - eccentricity)
    return result

In [11]:
my_semi_major = comet_table['Semi_Major_AU'].values * u.AU

my_semi_major

<Quantity [17.83414429,  2.21514129,  3.53465808,  3.83815916] AU>

In [12]:
my_ecc = comet_table['Eccentricity'].values * u.dimensionless_unscaled

my_ecc

<Quantity [0.96714291, 0.84833603, 0.751299  , 0.56961755]>

In [13]:
perihelion_AU = find_perihelion(my_semi_major, my_ecc)

perihelion_AU

<Quantity [0.58597811, 0.33595713, 0.879073  , 1.65187634] AU>

In [14]:
comet_table['Perihelion_AU'] = perihelion_AU

In [15]:
comet_table

,Name,Semi_Major_AU,Eccentricity,Semi_Major_km,Perihelion_AU
0,1P/Halley,17.834144,0.967143,2.667950e+09,0.585978
1,2P/Encke,2.215141,0.848336,3.313804e+08,0.335957
2,3D/Biela,3.534658,0.751299,5.287773e+08,0.879073
3,4P/Faye,3.838159,0.569618,5.741804e+08,1.651876


##### Save `comet_table` to a file (`.csv`)

In [16]:
comet_table.to_csv('./Data/Comet_DataFrame.csv', index=False)

---

# `DataFrames` and units

 * `DatFrames` and units do not play together well
 * Using a `DataFrame` and units requires you to:
   * Pull out `.values` and assign units everytime you want to use them.
   * `comet_table['Semi_Major_AU'].values * u.AU`
   * The to save your results, without units, back to the table
   * `comet_table['Perihelion_AU'] = perihelion_AU`

---

# Astropy `QTable`

* A `QTable` = a table with units!
* Does not have the huge number of `.methods` of a `DataFrame`
* Mostly **only** used by Astronomers
* Can be easily converted to a `DataFrame`

In [17]:
from astropy.table import QTable, Table

In [18]:
comet_table = QTable.read('./Data/Comets.csv', format='ascii.csv')

In [19]:
comet_table

Name,Semi_Major_AU,Eccentricity
str9,float64,float64
1P/Halley,17.8341442925537,0.967142908462304
2P/Encke,2.215141293583404,0.8483360282890077
3D/Biela,3.53465808340135,0.751299
4P/Faye,3.83815915788662,0.5696175496849397


#### Adding a unit to a column

In [20]:
comet_table['Semi_Major_AU'].unit = u.AU

In [21]:
comet_table

Name,Semi_Major_AU,Eccentricity
,AU,
str9,float64,float64
1P/Halley,17.8341442925537,0.967142908462304
2P/Encke,2.215141293583404,0.8483360282890077
3D/Biela,3.53465808340135,0.751299
4P/Faye,3.83815915788662,0.5696175496849397


In [22]:
comet_table['Semi_Major_AU']

<Quantity [17.83414429,  2.21514129,  3.53465808,  3.83815916] AU>

In [23]:
comet_table['Semi_Major_AU'].to(u.km)

<Quantity [2.66795001e+09, 3.31380421e+08, 5.28777323e+08, 5.74180437e+08] km>

In [24]:
comet_table['Semi_Major_AU'].unit

Unit("AU")

##### `QTables` use `.value` to access the value of a column (not the pandas `.values`)

In [25]:
comet_table['Semi_Major_AU'].value

array([17.83414429,  2.21514129,  3.53465808,  3.83815916])

In [26]:
def find_perihelion(semi_major, eccentricity):
    result = semi_major * (1.0 - eccentricity)
    return result

In [27]:
find_perihelion(comet_table['Semi_Major_AU'], comet_table['Eccentricity'])

<Quantity [0.58597811, 0.33595713, 0.879073  , 1.65187634] AU>

In [28]:
comet_table['Perihelion'] = find_perihelion(comet_table['Semi_Major_AU'], comet_table['Eccentricity'])

In [29]:
comet_table

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
1P/Halley,17.8341442925537,0.967142908462304,0.5859781115169156
2P/Encke,2.215141293583404,0.8483360282890077,0.3359571264858843
3D/Biela,3.53465808340135,0.751299,0.879072999999999
4P/Faye,3.83815915788662,0.5696175496849397,1.6518763430704317


In [30]:
comet_table['Perihelion'].to(u.km)

<Quantity [8.76610778e+07, 5.02584708e+07, 1.31507449e+08, 2.47117184e+08] km>

In [31]:
comet_table['Perihelion'].info.format = '.2f'

In [32]:
comet_table

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
1P/Halley,17.8341442925537,0.967142908462304,0.59
2P/Encke,2.215141293583404,0.8483360282890077,0.34
3D/Biela,3.53465808340135,0.751299,0.88
4P/Faye,3.83815915788662,0.5696175496849397,1.65


In [33]:
comet_table.info()

<QTable length=4>
     name      dtype  unit format  class  
------------- ------- ---- ------ --------
         Name    str9               Column
Semi_Major_AU float64   AU        Quantity
 Eccentricity float64               Column
   Perihelion float64   AU    .2f Quantity


In [34]:
for row in comet_table:
    output = f"The comet {row['Name']:9} has a peihelion distance of {row['Perihelion'].to(u.km):.4e}"
    print(output)

The comet 1P/Halley has a peihelion distance of 8.7661e+07 km
The comet 2P/Encke  has a peihelion distance of 5.0258e+07 km
The comet 3D/Biela  has a peihelion distance of 1.3151e+08 km
The comet 4P/Faye   has a peihelion distance of 2.4712e+08 km


## `QTable` manipulation and modification

* Does not have the huge number of `.methods` of a `DataFrame`
* Can do most 'obvious' stuff: slices, sorts, filtering, etc...
* Documentation: [Astropy Table Modifications](https://het.as.utexas.edu/HET/Software/Astropy-1.0/table/modify_table.html)

In [35]:
comet_table

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
1P/Halley,17.8341442925537,0.967142908462304,0.59
2P/Encke,2.215141293583404,0.8483360282890077,0.34
3D/Biela,3.53465808340135,0.751299,0.88
4P/Faye,3.83815915788662,0.5696175496849397,1.65


In [36]:
comet_table[0:2]

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
1P/Halley,17.8341442925537,0.967142908462304,0.59
2P/Encke,2.215141293583404,0.8483360282890077,0.34


In [37]:
comet_table[comet_table['Eccentricity'] < 0.8]

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
3D/Biela,3.53465808340135,0.751299,0.88
4P/Faye,3.83815915788662,0.5696175496849397,1.65


In [38]:
comet_table.sort('Perihelion')

In [39]:
comet_table

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
2P/Encke,2.215141293583404,0.8483360282890077,0.34
1P/Halley,17.8341442925537,0.967142908462304,0.59
3D/Biela,3.53465808340135,0.751299,0.88
4P/Faye,3.83815915788662,0.5696175496849397,1.65


In [40]:
comet_table[0:2]

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
2P/Encke,2.215141293583404,0.8483360282890077,0.34
1P/Halley,17.8341442925537,0.967142908462304,0.59


In [41]:
comet_table.sort('Perihelion', reverse=True)

comet_table

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
4P/Faye,3.83815915788662,0.5696175496849397,1.65
3D/Biela,3.53465808340135,0.751299,0.88
1P/Halley,17.8341442925537,0.967142908462304,0.59
2P/Encke,2.215141293583404,0.8483360282890077,0.34


### Can save `Qtables` with all the units info intact (`.ecsv`).

In [42]:
comet_table.write('./Data/Comet_QTable.ecsv', format='ascii.ecsv')

In [43]:
my_new_table = QTable.read('./Data/Comet_QTable.ecsv', format='ascii.ecsv')

In [44]:
my_new_table

Name,Semi_Major_AU,Eccentricity,Perihelion
,AU,,AU
str9,float64,float64,float64
4P/Faye,3.83815915788662,0.5696175496849397,1.65
3D/Biela,3.53465808340135,0.751299,0.88
1P/Halley,17.8341442925537,0.967142908462304,0.59
2P/Encke,2.215141293583404,0.8483360282890077,0.34


### Can convert `QTable` to pandas `DataFrame` - Loose all units info :(

In [45]:
comet_table_pandas = Table(comet_table).to_pandas()

In [46]:
comet_table_pandas

,Name,Semi_Major_AU,Eccentricity,Perihelion
0,4P/Faye,3.838159,0.569618,1.651876
1,3D/Biela,3.534658,0.751299,0.879073
2,1P/Halley,17.834144,0.967143,0.585978
3,2P/Encke,2.215141,0.848336,0.335957
